In [1]:
import os
from sklearn.preprocessing import LabelEncoder
import collections
import pandas as pd
import numpy as np

In [2]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [3]:
os.chdir("C:\\Users\\sridhar\\Desktop\\DS2\\ML course kaggle\\3 Decision trees")

In [4]:
# Create dataframe with dummy variables
def create_df(dic, feature_list):
    out = pd.DataFrame(dic)
    out = pd.concat([out, pd.get_dummies(out[feature_list])], axis = 1)
    out.drop(feature_list, axis = 1, inplace = True)
    return out

# Some feature values are present in train and absent in test and vice-versa.
def intersect_features(train, test):
    common_feat = list( set(train.keys()) & set(test.keys()))
    return train[common_feat], test[common_feat]

In [5]:
features = ['Looks', 'Alcoholic_beverage','Eloquence','Money_spent']

In [6]:
df_train = {}
df_train['Looks'] = ['handsome', 'handsome', 'handsome', 'repulsive',
                         'repulsive', 'repulsive', 'handsome'] 
df_train['Alcoholic_beverage'] = ['yes', 'yes', 'no', 'no', 'yes', 'yes', 'yes']
df_train['Eloquence'] = ['high', 'low', 'average', 'average', 'low',
                                   'high', 'average']
df_train['Money_spent'] = ['lots', 'little', 'lots', 'little', 'lots',
                                  'lots', 'lots']
df_train['Will_go'] = LabelEncoder().fit_transform(['+', '-', '+', '-', '-', '+', '+'])

data = create_df(df_train, features)
data

,Will_go,Looks_handsome,Looks_repulsive,Alcoholic_beverage_no,Alcoholic_beverage_yes,Eloquence_average,Eloquence_high,Eloquence_low,Money_spent_little,Money_spent_lots
0,0,1,0,0,1,0,1,0,0,1
1,1,1,0,0,1,0,0,1,1,0
2,0,1,0,1,0,1,0,0,0,1
3,1,0,1,1,0,1,0,0,1,0
4,1,0,1,0,1,0,0,1,0,1
5,0,0,1,0,1,0,1,0,0,1
6,0,1,0,0,1,1,0,0,0,1


In [7]:
data.shape

(7, 10)

1. What is the entropy of the  binary feature "Will_go" - 0 or 1 - two states in total

In [8]:
def entropy(labels):
    n_labels = len(labels)

    if n_labels <= 1:
        return 0

    counts = np.bincount(labels)
    probs = counts / n_labels
   
    

    entropy = 0

    # Compute standard entropy.
    for i in probs:
        entropy -= i * np.log2(i)

    return ('Entropy:',entropy)


In [9]:
entropy(data['Will_go'])

('Entropy:', 0.9852281360342515)

In [10]:
print("Entropy of Will_do:{}".format(-((3/7)*np.log2(3/7)+(4/7)*np.log2(4/7))))

Entropy of Will_do:0.9852281360342515


2. Let's split the data by the feature "Looks_handsome".
   What is the entropy S1 of the left group - the one with "Looks_handsome". What is the entropy S2 in the opposite group? What is the information gain (IG) if we consider such a split?

In [11]:
entropy(data['Looks_handsome'])

('Entropy:', 0.9852281360342515)

Train a decision tree using sklearn on the training data.

In [12]:
y=data['Will_go']
x=data.drop('Will_go', axis=1)

In [13]:
Tree=DecisionTreeClassifier(criterion='entropy')
Tree.fit(x,y)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

Display the resulting tree using graphviz

In [14]:
dot_data=export_graphviz(Tree, out_file='tree.dot', feature_names=x.columns)  

Implement a function to calculate the Shannon Entropy

In [15]:
def entropy(labels):
    n_labels = len(labels)

    if n_labels <= 1:
        return 0

    counts = np.bincount(labels)
    probs = counts / n_labels
   
    

    entropy = 0

    # Compute standard entropy.
    for i in probs:
        entropy -= i * np.log2(i)

    return (entropy)

Consider the following warm-up example: we have 9 blue balls and 11 yellow balls. Let ball have label 1 if it is blue, 0 otherwise.

In [16]:
balls = [1 for i in range(9)] + [0 for i in range(11)]

In [17]:
balls_left  = [1 for i in range(8)] + [0 for i in range(5)] # 8 blue and 5 yellow
balls_right = [1 for i in range(1)] + [0 for i in range(6)] # 1 blue and 6 yellow

In [18]:
entropy(balls)

0.9927744539878083

In [19]:
entropy(balls_left)

0.9612366047228759

In [20]:
entropy(balls_right)

0.5916727785823275

 What is the entropy of a fair dice? (where we look at a dice as a system with 6 equally probable states)?

In [21]:
-(1/6*(np.log2(1/6))+(5/6*(np.log2(5/6))))

0.6500224216483541

 What is the information gain from splitting the initial dataset into balls_left and balls_right ?

In [22]:
entropy(balls)-(13/20*entropy(balls_left))-(7/20*entropy(balls_right))

0.16088518841412436

# Decisin tree

In [23]:
train=pd.read_csv("adult_train.csv")
train.head()

,Age,Workclass,fnlwgt,Education,Education_Num,Martial_Status,Occupation,Relationship,Race,Sex,Capital_Gain,Capital_Loss,Hours_per_week,Country,Target
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [24]:
train.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Age,32561,NaN,NaN,NaN,38.5816,13.6404,17,28,37,48,90
Workclass,30725,8,Private,22696,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fnlwgt,32561,NaN,NaN,NaN,189778,105550,12285,117827,178356,237051,1.48470e+06
Education,32561,16,HS-grad,10501,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Education_Num,32561,NaN,NaN,NaN,10.0807,2.57272,1,9,10,12,16
Martial_Status,32561,7,Married-civ-spouse,14976,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Occupation,30718,14,Prof-specialty,4140,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Relationship,32561,6,Husband,13193,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Race,32561,5,White,27816,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sex,32561,2,Male,21790,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
test=pd.read_csv("adult_test.csv")
test.head()

,Age,Workclass,fnlwgt,Education,Education_Num,Martial_Status,Occupation,Relationship,Race,Sex,Capital_Gain,Capital_Loss,Hours_per_week,Country,Target
0,|1x3 Cross validator,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,25,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K.
2,38,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K.
3,28,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K.
4,44,Private,160323.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K.


In [26]:
test.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Age,16282,74,35,461,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Workclass,15318,8,Private,11210,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fnlwgt,16281,NaN,NaN,NaN,189436,105715,13492,116736,177831,238384,1.4904e+06
Education,16281,16,HS-grad,5283,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Education_Num,16281,NaN,NaN,NaN,10.0729,2.56755,1,9,10,12,16
Martial_Status,16281,7,Married-civ-spouse,7403,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Occupation,15315,14,Prof-specialty,2032,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Relationship,16281,6,Husband,6523,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Race,16281,5,White,13946,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sex,16281,2,Male,10860,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# encode target variable as integer
train['Target'].replace([' <=50K',' >50K'],[0,1], inplace=True)

test['Target'].replace([' <=50K.',' >50K.'],[0,1], inplace=True)

In [28]:
train.dtypes

Age                int64
Workclass         object
fnlwgt             int64
Education         object
Education_Num      int64
Martial_Status    object
Occupation        object
Relationship      object
Race              object
Sex               object
Capital_Gain       int64
Capital_Loss       int64
Hours_per_week     int64
Country           object
Target             int64
dtype: object

In [29]:
test.dtypes

Age                object
Workclass          object
fnlwgt            float64
Education          object
Education_Num     float64
Martial_Status     object
Occupation         object
Relationship       object
Race               object
Sex                object
Capital_Gain      float64
Capital_Loss      float64
Hours_per_week    float64
Country            object
Target            float64
dtype: object

In [30]:
test.drop(0,inplace=True)

In [31]:
test['Age'] = test['Age'].astype(int)
test['fnlwgt'] = test['fnlwgt'].astype(int)
test['Education_Num'] = test['Education_Num'].astype(int)
test['Capital_Gain'] = test['Capital_Gain'].astype(int)
test['Capital_Loss'] = test['Capital_Loss'].astype(int)
test['Hours_per_week'] = test['Hours_per_week'].astype(int)
test['Target']=test['Target'].astype(int)

In [32]:
# choose categorical and continuous features from data

categorical_columns = [c for c in train.columns 
                       if train[c].dtype.name == 'object']
numerical_columns = [c for c in train.columns 
                     if train[c].dtype.name != 'object']

In [33]:
# fill missing data

for c in categorical_columns:
    train[c].fillna(train[c].mode(), inplace=True)
    test[c].fillna(train[c].mode(), inplace=True)
    
for c in numerical_columns:
    train[c].fillna(train[c].median(), inplace=True)
    test[c].fillna(train[c].median(), inplace=True)

In [34]:
train = pd.concat([train[numerical_columns],
    pd.get_dummies(train[categorical_columns])], axis=1)

test = pd.concat([test[numerical_columns],
    pd.get_dummies(test[categorical_columns])], axis=1)

In [35]:
train.shape, test.shape

((32561, 106), (16281, 105))

In [36]:
set(train.columns) - set(test.columns)

{'Country_ Holand-Netherlands'}

In [37]:
test['Country_ Holand-Netherlands'] = 0

In [38]:
x_train = train.drop(['Target'], axis=1)
y_train = train['Target']

x_test = test.drop(['Target'], axis=1)
y_test = test['Target']

Train a decision tree (DecisionTreeClassifier) with a maximum depth of 3, and evaluate the accuracy metric on the test data. Use parameter random_state = 17 for results reproducibility.

In [39]:
tree=DecisionTreeClassifier(max_depth=3, random_state=17)

In [40]:
tree.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=17,
            splitter='best')

In [41]:
# Prediction
tree_predictions = tree.predict(x_test)

In [42]:
# Accuracy score
accuracy_score(tree_predictions,y_test)

0.8447884036607088

Train a decision tree (DecisionTreeClassifier, random_state = 17). Find the optimal maximum depth using 5-fold cross-validation (GridSearchCV).

In [43]:
tree_para = {'max_depth':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]}
grid = GridSearchCV(DecisionTreeClassifier(random_state=17), tree_para, cv=5)
grid.fit(x_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=17,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [44]:
grid.best_params_

{'max_depth': 9}

In [45]:
grid.best_score_

0.8562697705844415

Train a decision tree with best maximum depth , and compute the test set accuracy. Use parameter random_state = 17 for reproducibility.

In [46]:
tuned_tree= DecisionTreeClassifier(max_depth=9, random_state=17)
tuned_tree.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=9,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=17,
            splitter='best')

In [47]:
tuned_tree_predictions = tuned_tree.predict(x_test)

In [48]:
accuracy_score(tuned_tree_predictions, y_test)

0.8471838339168356

Random forest

Train a random forest (RandomForestClassifier). Set the number of trees to 100 and use random_state = 17.

In [49]:
rf =RandomForestClassifier(n_estimators=100, random_state=17)
rf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=17, verbose=0, warm_start=False)

In [50]:
rf_predict=rf.predict(x_test)

In [51]:
accuracy_score(rf_predict, y_test)

0.8576254529820035

In [52]:
c=list(range(test.shape[1]))

Train a random forest (RandomForestClassifier). Tune the maximum depth and maximum number of features for each tree using GridSearchCV

In [53]:
rf_param={'max_depth': range(10, 21),'max_features': range(5, 105, 20)}
rf_grid=GridSearchCV(RandomForestClassifier(random_state=17), rf_param, cv=5)

In [54]:
rf_grid.fit(x_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=17, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': range(10, 21), 'max_features': range(5, 105, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [55]:
rf_grid.best_params_

{'max_depth': 12, 'max_features': 65}

In [56]:
tuned_rf=RandomForestClassifier(n_estimators=100, max_depth=12, max_features=65, random_state=17)
tuned_rf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=12, max_features=65, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=17, verbose=0, warm_start=False)

In [57]:
tuned_rf_predict=tuned_rf.predict(x_test)

In [58]:
accuracy_score(tuned_rf_predict, y_test)

0.865118850193477